In [3]:
from graphviz import Digraph
from tabulate import tabulate

class NFA:
    def __init__(self, states, alphabet, transitions, initial_state, accepting_states):
        self.states = states
        self.alphabet = alphabet
        self.transitions = transitions
        self.initial_state = initial_state
        self.accepting_states = accepting_states

def epsilon_closure(states, nfa, visited=None):
    if visited is None:
        visited = set()

    closure = set(states)

    for state in states:
        if state not in visited:
            visited.add(state)
            if 'e' in nfa.transitions.get(state, {}):
                epsilon_states = nfa.transitions[state]['e']
                closure |= epsilon_closure(epsilon_states, nfa, visited)

    return closure

def get_new_state(states):
    return tuple(sorted(states))

def convert_nfa_to_dfa(nfa):
    dfa = {}
    dfa_transition_table = []

    start_state = epsilon_closure([nfa.initial_state], nfa)
    start_state = get_new_state(start_state)
    dfa[start_state] = {}

    queue = [start_state]

    symbols = set(nfa.alphabet) - {'e'}

    while queue:
        current_state = queue.pop(0)

        for symbol in symbols:
            new_state = epsilon_closure([trans_state for state in current_state for trans_state in nfa.transitions.get(state, {}).get(symbol, set())], nfa)

            if not new_state:
                continue

            new_state = get_new_state(new_state)

            if new_state not in dfa:
                dfa[new_state] = {}
                queue.append(new_state)

            dfa[current_state][symbol] = new_state

    dfa_states = list(dfa.keys())

    for state in dfa_states:
        row = [state]

        for symbol in symbols:
            row.append(dfa[state].get(symbol, set()))

        dfa_transition_table.append(row)

    dfa_accepting_states = [state for state in dfa_states if any(accept_state in state for accept_state in nfa.accepting_states)]

    return NFA(dfa_states, nfa.alphabet, dfa, start_state, dfa_accepting_states)

# Get NFA input from the user
nfa_states = input("Enter NFA states (comma-separated): ").split(',')
nfa_alphabet = input("Enter NFA alphabet (comma-separated): ").split(',')
nfa_transitions = {}
transition_input = ""
while transition_input != 'done':
    transition_input = input("Enter NFA transition (state,symbol):next_state (or enter 'done' to finish): ")
    if transition_input == 'done':
        break

    transition_parts = transition_input.split(',')
    if len(transition_parts) != 3:
        print("Invalid input format. Please try again.")
        continue

    state, symbol, next_state = transition_parts
    nfa_transitions.setdefault(state, {}).setdefault(symbol, set()).add(next_state)

nfa_initial_state = input("Enter NFA initial state: ")
nfa_accepting_states = input("Enter NFA accepting states (comma-separated): ").split(',')

nfa = NFA(nfa_states, nfa_alphabet, nfa_transitions, nfa_initial_state, nfa_accepting_states)

# Convert NFA to DFA
dfa = convert_nfa_to_dfa(nfa)

# Find DFA states containing NFA accepting states
accepting_dfa_states = []
for dfa_state, nfa_states in dfa.transitions.items():
    if any(nfa_accepting_state in nfa_states for nfa_accepting_state in nfa.accepting_states):
        accepting_dfa_states.append(dfa_state)

# Print the transition table of NFA and DFA
print("\nTransition Table of NFA:")
print(tabulate([(state, nfa.transitions.get(state, {})) for state in nfa.states], headers=['State', 'Transitions']))

print("\nTransition Table of DFA:")
dfa_table_with_markers = []
start_state_marker = '->'
accepting_state_marker = '*'

# Mark the start state and accepting states in the DFA transition table
for row_state, row_transitions in dfa.transitions.items():
    new_row = [start_state_marker + str(row_state) if row_state == dfa.initial_state else str(row_state)]
    for symbol in nfa.alphabet:
        target_state = row_transitions.get(symbol, set())
        target_state_str = str(target_state)
        if target_state in dfa.accepting_states:
            target_state_str += accepting_state_marker
        new_row.append(f"{symbol}:{target_state_str}")
    dfa_table_with_markers.append(new_row)

print(tabulate(dfa_table_with_markers, headers=['State'] + list(nfa.alphabet)))

# Visualize the DFA as a graph
dot = Digraph(comment='DFA Graph')
for state, transitions in dfa.transitions.items():
    if state == get_new_state(epsilon_closure([nfa.initial_state], nfa)):
        dot.node(str(state), label=start_state_marker + str(state))
    elif state in dfa.accepting_states:
        dot.node(str(state), label=str(state) + accepting_state_marker)
    else:
        dot.node(str(state), label=str(state))

    for symbol, target_state in transitions.items():
        dot.edge(str(state), str(target_state), label=symbol)

dot.render('dfa_graph', format='png', cleanup=True)

# Print the start state and accepting states in the DFA
print("\nStart State in DFA: ", dfa.initial_state)
print("Accepting States in DFA: ", dfa.accepting_states)

Enter NFA states (comma-separated): q0,q1,q2
Enter NFA alphabet (comma-separated): a,b
Enter NFA transition (state,symbol):next_state (or enter 'done' to finish): q0,b,q0
Enter NFA transition (state,symbol):next_state (or enter 'done' to finish): q0,e,q1
Enter NFA transition (state,symbol):next_state (or enter 'done' to finish): q1,a,q1
Enter NFA transition (state,symbol):next_state (or enter 'done' to finish): q1,e,q2
Enter NFA transition (state,symbol):next_state (or enter 'done' to finish): q2,a,q1
Enter NFA transition (state,symbol):next_state (or enter 'done' to finish): q2,b,q0
Enter NFA transition (state,symbol):next_state (or enter 'done' to finish): done
Enter NFA initial state: q0
Enter NFA accepting states (comma-separated): q2

Transition Table of NFA:
State    Transitions
-------  --------------------------
q0       {'b': {'q0'}, 'e': {'q1'}}
q1       {'a': {'q1'}, 'e': {'q2'}}
q2       {'a': {'q1'}, 'b': {'q0'}}

Transition Table of DFA:
State                 a           